In [14]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import vgg16_bn, VGG16_BN_Weights

### Model 1 and 3: 2D and VGG-16

In [ ]:


class HyperspectralTransferCNN(nn.Module):
    def __init__(self, input_bands=63, num_classes=3, backbone_type='vgg16'):
        super(HyperspectralTransferCNN, self).__init__()

        # Step 1: Reduce hyperspectral input to 3 channels
        self.mapping = nn.Conv2d(input_bands, 3, kernel_size=1)

        # Step 2: Backbone
        if backbone_type == 'vgg16': # VGG-16 model
            backbone = models.vgg16_bn(pretrained=True)
            self.features = backbone.features
            self.blocks = self.features  # Alias to make training logic consistent
            in_features = 512

        elif backbone_type == 'resnet34': # Not tested
            backbone = models.resnet34(pretrained=True)
            self.features = nn.Sequential(*list(backbone.children())[:-2])
            in_features = 512
        elif backbone_type == 'custom': # 2D model
            self.features = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(32, 64, kernel_size=3, padding=1),
                nn.ReLU()
            )
            self.blocks = self.features  # Alias to make training logic consistent
            in_features = 64  # Output channels from last conv layer
        else:
            raise ValueError("Unsupported backbone. Choose 'vgg16', 'resnet34', or 'custom'.")

        # Step 3: Global pooling (works for any input size)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        # Step 4: Classification head
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.mapping(x)
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x


### Model 2: Hybrid with Projection Matrix

In [ ]:
class ImprovedHybrid3D2DCNN_v2(nn.Module):
    def __init__(self, in_bands=63, num_classes=3):
        super().__init__()

        # Spectral–spatial 3D encoder
        self.encoder3d = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=(3, 3, 3), padding=1),
            nn.BatchNorm3d(16), nn.ReLU(inplace=True),
            nn.Conv3d(16, 32, kernel_size=(3, 3, 3), padding=1, stride=(2, 1, 1)),
            nn.BatchNorm3d(32), nn.ReLU(inplace=True),
            nn.Dropout3d(0.1),
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1, stride=(2, 1, 1)),
            nn.BatchNorm3d(64), nn.ReLU(inplace=True),
            nn.Dropout3d(0.1),
            nn.AdaptiveAvgPool3d((1, None, None))  # retain spatial, compress spectral
        )

        # Flatten spectral dim
        self.project_2d = nn.Conv2d(64, 32, kernel_size=1)

        # 2D convolutional head (deeper)
        self.features2d = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.Dropout2d(0.2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.Dropout2d(0.2),
        )
        # self.blocks = self.features2d  # Alias to make training logic consistent

        # Optional self-attention (can remove for simplicity)
        self.attention = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Conv2d(128, 32, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 128, kernel_size=1),
            nn.Sigmoid()
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x: (B, Bands, H, W)
        x = x.unsqueeze(1)            # (B,1,Bands,H,W)
        x = self.encoder3d(x)         # (B,64,1,H,W)
        x = x.squeeze(2)              # (B,64,H,W)
        x = self.project_2d(x)        # (B,32,H,W)
        x = self.features2d(x)        # (B,128,H,W)

        # Self-attention (optional)
        attn = self.attention(x)      # (B,128,1,1)
        x = x * attn                  # Channel-wise attention

        x = self.pool(x)              # (B,128,1,1)
        return self.classifier(x)     # (B,num_classes)


### Model 2: Hybrid with Mapping Layer

In [ ]:
# class ImprovedHybrid3D2DCNN_v3(nn.Module):
#     def __init__(self, in_bands=47, num_classes=3, mapped_bands=63):
#         super().__init__()
#         self.in_bands = in_bands
#         self.mapped_bands = mapped_bands

#         # Learned spectral mapping: projects any-band input to 63-band space
#         self.mapping = nn.Linear(in_bands, mapped_bands)

#         # Spectral–spatial 3D encoder (expecting mapped_bands as spectral depth)
#         self.encoder3d = nn.Sequential(
#             nn.Conv3d(1, 16, kernel_size=(3, 3, 3), padding=1),
#             nn.BatchNorm3d(16), nn.ReLU(inplace=True),
#             nn.Conv3d(16, 32, kernel_size=(3, 3, 3), padding=1, stride=(2, 1, 1)),
#             nn.BatchNorm3d(32), nn.ReLU(inplace=True),
#             nn.Dropout3d(0.1),
#             nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1, stride=(2, 1, 1)),
#             nn.BatchNorm3d(64), nn.ReLU(inplace=True),
#             nn.Dropout3d(0.1),
#             nn.AdaptiveAvgPool3d((1, None, None))  # retain spatial, compress spectral
#         )

#         # Flatten spectral dim
#         self.project_2d = nn.Conv2d(64, 32, kernel_size=1)

#         # 2D convolutional head
#         self.features2d = nn.Sequential(
#             nn.Conv2d(32, 64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(64), nn.ReLU(inplace=True),
#             nn.Dropout2d(0.2),
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(64), nn.ReLU(inplace=True),
#             nn.Conv2d(64, 128, kernel_size=3, padding=1),
#             nn.BatchNorm2d(128), nn.ReLU(inplace=True),
#             nn.Dropout2d(0.2),
#         )

#         # Optional self-attention
#         self.attention = nn.Sequential(
#             nn.AdaptiveAvgPool2d((1, 1)),
#             nn.Conv2d(128, 32, kernel_size=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(32, 128, kernel_size=1),
#             nn.Sigmoid()
#         )

#         self.pool = nn.AdaptiveAvgPool2d((1, 1))

#         self.classifier = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(128, 64),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.5),
#             nn.Linear(64, num_classes)
#         )

#     def forward(self, x):
#         # x: (B, Bands, H, W) — raw input
#         B, C, H, W = x.shape  # C = in_bands

#         # 1. Spectral mapping: (B, C, H, W) → (B, 63, H, W)
#         x = x.permute(0, 2, 3, 1).reshape(-1, C)        # (B*H*W, in_bands)
#         x = self.mapping(x)                            # (B*H*W, mapped_bands)
#         x = x.view(B, H, W, self.mapped_bands).permute(0, 3, 1, 2)  # (B, mapped_bands, H, W)

#         # 2. Continue normal flow
#         x = x.unsqueeze(1)            # (B, 1, mapped_bands, H, W)
#         x = self.encoder3d(x)         # (B, 64, 1, H, W)
#         x = x.squeeze(2)              # (B, 64, H, W)
#         x = self.project_2d(x)        # (B, 32, H, W)
#         x = self.features2d(x)        # (B, 128, H, W)

#         attn = self.attention(x)      # (B, 128, 1, 1)
#         x = x * attn                  # attention-weighted features

#         x = self.pool(x)              # (B, 128, 1, 1)
#         return self.classifier(x)     # (B, num_classes)


### Model 4: VGG-16 sSE 

In [6]:
class SpatialSEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SpatialSEBlock, self).__init__()
        self.fc1 = nn.Linear(in_channels, in_channels // reduction)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels)

    def forward(self, x):
        B, C, H, W = x.size()
        squeeze = F.adaptive_avg_pool2d(x, 1).view(B, C)
        excitation = torch.sigmoid(self.fc2(F.relu(self.fc1(squeeze)))).view(B, C, 1, 1)
        return x * excitation


class VGG16WithAttention(nn.Module):
    def __init__(self, input_bands=63, num_classes=3, pretrained=True):
        super().__init__()

        # 1x1 conv to reduce hyperspectral to 3-channel RGB equivalent
        self.mapping = nn.Conv2d(input_bands, 3, kernel_size=1)

        # Load pretrained VGG16 backbone
        vgg = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1 if pretrained else None)
        layers = list(vgg.features.children())

        # Divide into 5 blocks as in VGG
        self.blocks = nn.ModuleList()
        self.attentions = nn.ModuleList()
        self.pooling = nn.ModuleList()

        cfg = [6, 13, 23, 33, 43]  # Layer indices for MaxPool
        in_channels_list = [64, 128, 256, 512, 512]

        prev = 0
        for idx, end in enumerate(cfg):
            block = nn.Sequential(*layers[prev:end - 1])  # up to before MaxPool
            self.blocks.append(block)
            self.attentions.append(SpatialSEBlock(in_channels_list[idx]))
            self.pooling.append(layers[end - 1])  # the MaxPool itself
            prev = end

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.mapping(x)  # From hyperspectral input to 3-channel

        for block, attn, pool in zip(self.blocks, self.attentions, self.pooling):
            x = block(x)
            x = attn(x)  # Attention after block, before pooling
            x = pool(x)

        x = self.pool(x)
        x = self.classifier(x)
        return x


### Model 5: VGG-16 CBAM with Projection Matrix

In [ ]:
class CBAM(nn.Module):
    def __init__(self, channels, reduction_ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        # Channel attention
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction_ratio, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction_ratio, channels, bias=False)
        )
        self.sigmoid_channel = nn.Sigmoid()

        # Spatial attention
        self.conv_spatial = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2)
        self.sigmoid_spatial = nn.Sigmoid()

    def forward(self, x):
        # Channel Attention
        b, c, _, _ = x.size()
        avg_out = self.fc(self.avg_pool(x).view(b, c))
        max_out = self.fc(self.max_pool(x).view(b, c))
        scale = self.sigmoid_channel(avg_out + max_out).view(b, c, 1, 1)
        x = x * scale

        # Spatial Attention
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = x * self.sigmoid_spatial(self.conv_spatial(torch.cat([avg_out, max_out], dim=1)))
        return x


class SpectralSpatialInputBlock(nn.Module):
    def __init__(self, in_channels=47, mid_channels=16):
        super(SpectralSpatialInputBlock, self).__init__()
        self.mapping = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.mapping(x)


class VGG16WithCBAM(nn.Module):
    def __init__(self, num_classes=3, in_channels=47):
        super(VGG16WithCBAM, self).__init__()

        self.input_block = SpectralSpatialInputBlock(in_channels=in_channels, mid_channels=16)

        def make_block(in_c, out_c, num_convs):
            layers = []
            for _ in range(num_convs):
                layers.append(nn.Conv2d(in_c, out_c, kernel_size=3, padding=1))
                layers.append(nn.BatchNorm2d(out_c))
                layers.append(nn.ReLU(inplace=True))
                in_c = out_c
            layers.append(CBAM(out_c))
            return nn.Sequential(*layers)

        self.block1 = make_block(16, 64, 2)
        self.block2 = make_block(64, 128, 2)
        self.block3 = make_block(128, 256, 3)
        self.block4 = make_block(256, 512, 3)
        self.block5 = make_block(512, 512, 3)

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.input_block(x)
        x = self.block1(x)
        x = F.max_pool2d(x, 2)
        x = self.block2(x)
        x = F.max_pool2d(x, 2)
        x = self.block3(x)
        x = F.max_pool2d(x, 2)
        x = self.block4(x)
        x = F.max_pool2d(x, 2)
        x = self.block5(x)
        x = F.max_pool2d(x, 2)
        x = self.pool(x)
        return self.classifier(x)

### Model 5: VGG-16 CBAM with Mapping Layer

In [ ]:

# class CBAM(nn.Module):
#     def __init__(self, channels, reduction_ratio=16, kernel_size=7):
#         super(CBAM, self).__init__()
#         # Channel attention
#         self.avg_pool = nn.AdaptiveAvgPool2d(1)
#         self.max_pool = nn.AdaptiveMaxPool2d(1)
#         self.fc = nn.Sequential(
#             nn.Linear(channels, channels // reduction_ratio, bias=False),
#             nn.ReLU(inplace=True),
#             nn.Linear(channels // reduction_ratio, channels, bias=False)
#         )
#         self.sigmoid_channel = nn.Sigmoid()

#         # Spatial attention
#         self.conv_spatial = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2)
#         self.sigmoid_spatial = nn.Sigmoid()

#     def forward(self, x):
#         # Channel Attention
#         b, c, _, _ = x.size()
#         avg_out = self.fc(self.avg_pool(x).view(b, c))
#         max_out = self.fc(self.max_pool(x).view(b, c))
#         scale = self.sigmoid_channel(avg_out + max_out).view(b, c, 1, 1)
#         x = x * scale

#         # Spatial Attention
#         avg_out = torch.mean(x, dim=1, keepdim=True)
#         max_out, _ = torch.max(x, dim=1, keepdim=True)
#         x = x * self.sigmoid_spatial(self.conv_spatial(torch.cat([avg_out, max_out], dim=1)))
#         return x


# class SpectralSpatialInputBlock(nn.Module):
#     def __init__(self, in_channels=47, mid_channels=16):
#         super(SpectralSpatialInputBlock, self).__init__()
#         self.mapping = nn.Sequential(
#             nn.Conv2d(in_channels, mid_channels, kernel_size=1),
#             nn.BatchNorm2d(mid_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(mid_channels, mid_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(mid_channels),
#             nn.ReLU(inplace=True),
#         )

#     def forward(self, x):
#         return self.mapping(x)


# class VGG16WithCBAM(nn.Module):
#     def __init__(self, num_classes=3, in_channels=47):  # Default in_channels = 47
#         super(VGG16WithCBAM, self).__init__()

#         # Define the projection layer to reduce channels from other values to 47
#         self.input_projection = nn.Conv2d(in_channels, 47, kernel_size=1) if in_channels != 47 else None

#         # Define the rest of the model
#         self.input_block = SpectralSpatialInputBlock(in_channels=47, mid_channels=16)

#         def make_block(in_c, out_c, num_convs):
#             layers = []
#             for _ in range(num_convs):
#                 layers.append(nn.Conv2d(in_c, out_c, kernel_size=3, padding=1))
#                 layers.append(nn.BatchNorm2d(out_c))
#                 layers.append(nn.ReLU(inplace=True))
#                 in_c = out_c
#             layers.append(CBAM(out_c))
#             return nn.Sequential(*layers)

#         self.block1 = make_block(16, 64, 2)
#         self.block2 = make_block(64, 128, 2)
#         self.block3 = make_block(128, 256, 3)
#         self.block4 = make_block(256, 512, 3)
#         self.block5 = make_block(512, 512, 3)

#         self.pool = nn.AdaptiveAvgPool2d((1, 1))
#         self.classifier = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(256, num_classes)
#         )

#     def forward(self, x):
#         # If input_channels is not 47, apply the 1x1 convolution to project input channels
#         if self.input_projection is not None:
#             x = self.input_projection(x)

#         # Pass through the input block and subsequent layers
#         x = self.input_block(x)
#         x = self.block1(x)
#         x = F.max_pool2d(x, 2)
#         x = self.block2(x)
#         x = F.max_pool2d(x, 2)
#         x = self.block3(x)
#         x = F.max_pool2d(x, 2)
#         x = self.block4(x)
#         x = F.max_pool2d(x, 2)
#         x = self.block5(x)
#         x = F.max_pool2d(x, 2)
#         x = self.pool(x)
#         return self.classifier(x)